In [1]:
import mbuild as mb

import gmso
from gmso import ForceField
from gmso.external import convert_mbuild
from gmso.formats.lammpsdata import write_lammpsdata

/Users/raymatsumoto/foyer/foyer/forcefield.py:433: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/raymatsumoto/foyer/foyer/forcefield.py:445: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'


In [2]:
# Load in single water TIP3P structure

water = mb.load("files/tip3p.mol2")
water = water.children[0]
water.name = "water"

# element_map = which site name corresponds to which atom_type name. 
# In the future atomtyping will be done through foyer. 
element_map = {"O": "opls_111",
               "H": "opls_112"}

In [3]:
# Fill a box with 1000 water molecule

water_box = mb.fill_box(
    compound=water,     
    n_compounds=1000,
    density=1000,
)

In [4]:
# Load in topology forcefield

forcefield = ForceField("files/tip3p.xml")

In [5]:
# Generate a topology from the mbuild compound

top = convert_mbuild.from_mbuild(water_box)

/Users/raymatsumoto/installs/gmso/gmso/core/subtopology.py:64: UserWarning: Setting a parent is potentially dangerous. Consider using Topology.add_subtopology instead
  'Setting a parent is potentially dangerous. Consider using '
/Users/raymatsumoto/installs/gmso/gmso/core/element.py:60: UserWarning: Numbers and spaces are not considered when searching by element symbol. 

             {} became {}'.format(symbol, symbol_trimmed)
  warnings.warn(msg)
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site O, id 4837508264>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4849587928>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4849588656>
  warnings.warn('Non-param

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4852233048>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4852233272>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site O, id 4852233608>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4852233496>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:516: UserWarning: Non-parametrized site detected <Site H, id 4852233720>
  warnings.warn('Non-parametrized site detected {}'.format(site))
/Users/raymatsumoto/insta

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849586304, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4844883864, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4844883808, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4844883920, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849480144, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849480200, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849480368, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849480256, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849400240, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849400296, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849400464, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4849400352, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

...

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4848158200, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4848158256, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4848094176, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4848158312, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847936568, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847936624, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847936680, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847936792, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847968888, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847968944, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847935672, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847969000, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847538304, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847538360, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847538416, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847971632, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847541720, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847541776, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847541832, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/core/topology.py:477: UserWarning: Non-parametrized Connection <2-partner Connection, id 4847541944, type None, name Bond> detected
  warnings.warn('Non-parametrized Connection {} detected'.format(c))
/Users/raymatsumoto/installs/gmso/gmso/c

In [6]:
# Assign atom types
for atom in top.sites:
    atom.atom_type = forcefield.atom_types[element_map[atom.name]]
    
# Assign bond types
for bond in top.bonds:
    bond.bond_type = bond.connection_type = forcefield.bond_types["opls_111~opls_112"]

# Create angles with correct atom type and add to top
# Adding `connection_members` in order specified by name
for subtop in top.subtops:
    angle = gmso.core.angle.Angle(
        connection_members=[subtop.sites[1], subtop.sites[0], subtop.sites[2]],
        name="opls_112~opls_111~opls_112",
        connection_type=forcefield.angle_types["opls_112~opls_111~opls_112"]
    )
    top.add_connection(angle, update_types=False)

top.update_topology()

In [7]:
# Write out LAMMPS Files
write_lammpsdata(top, 'data.tip3p')

/Users/raymatsumoto/installs/gmso/gmso/formats/lammpsdata.py:82: UserWarning: Orthorhombic box detected
  warnings.warn("Orthorhombic box detected")


In [8]:
# Run the lammps simulation
!lmp_serial < files/in.tip3p

/bin/sh: lmp_serial: command not found
